In [2]:
!pip install -q kaggle

In [3]:
# Importar librerías
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [4]:
# Verificar GPU
print("\nGPU disponible:", "Sí" if tf.config.list_physical_devices('GPU') else "No")


GPU disponible: Sí


In [4]:
# %% [markdown]
# **Paso 2: Cargar el dataset**
# @title Cargar dataset
# Configurar tu URL aquí (formato .zip)
URL_PONER = "https://tudataset.com/archivo.zip"  # <--- REEMPLAZAR CON TU URL



--2025-02-28 18:14:22--  https://tudataset.com/archivo.zip
Resolving tudataset.com (tudataset.com)... failed: Name or service not known.
wget: unable to resolve host address ‘tudataset.com’


BadZipFile: File is not a zip file

In [ ]:
# Descargar y descomprimir dataset
!wget --no-check-certificate "{URL_PONER}" -O dataset.zip
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [ ]:
# %% [markdown]
# **Paso 3: Preparar generadores de datos**
# @title Preparar datos
# Configuración aumentación de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Parámetros
img_size = (224, 224)
batch_size = 32

# Generadores
train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:

# %% [markdown]
# **Paso 4: Construir el modelo**
# @title Construir modelo
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# %% [markdown]
# **Paso 5: Entrenamiento**
# @title Entrenar modelo
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy')
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop]
)


In [ ]:



# %% [markdown]
# **Paso 6: Resultados**
# @title Mostrar resultados
# Gráfico de precisión
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend()





In [ ]:
# Gráfico de pérdida
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida del modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend()
plt.show()

In [ ]:
# Guardar modelo final
model.save('modelo_reciclaje.h5')
print("Modelo guardado en modelo_reciclaje.h5")



In [ ]:
# Descargar archivos desde Colab
from google.colab import files
files.download('modelo_reciclaje.h5')